Projet Prediction
SEARLE Oliver B11

In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import xgboost as xgb
import warnings


In [15]:
#Train data
train_home = pd.read_csv('X_Train_Data/train_home_team_statistics_df.csv',index_col=0)
train_away = pd.read_csv('X_Train_Data/train_away_team_statistics_df.csv',index_col=0)
y_train_score = pd.read_csv('Y_train_1rknArQ.csv',index_col=0)

#Test data
test_home = pd.read_csv('X_Test_Data/test_home_team_statistics_df.csv',index_col=0)
test_away = pd.read_csv('X_Test_Data/test_away_team_statistics_df.csv',index_col=0)

In [12]:
# Supprime 2 premières Colonnes
train_home = train_home.iloc[:,2:]
train_away = train_away.iloc[:,2:]

train_home.columns = 'HOME_' + train_home.columns
train_away.columns = 'AWAY_' + train_away.columns

test_home.columns = 'HOME_' + test_home.columns
test_away.columns = 'AWAY_' + test_away.columns

# Combine les donnés home et away
train_data = pd.concat([train_home, train_away], join='inner', axis=1)
test_data = pd.concat([test_home, test_away], join='inner', axis=1)

# Drop les lignes avec valeurs manquantes
train_data.dropna(inplace=True)

# Alligne les lignes y_train_score et train_data
train_scores = y_train_score.loc[train_data.index]


In [13]:
train_data.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,AWAY_TEAM_YELLOWCARDS_5_last_match_std,AWAY_TEAM_REDCARDS_5_last_match_std,AWAY_TEAM_OFFSIDES_5_last_match_std,AWAY_TEAM_ATTACKS_5_last_match_std,AWAY_TEAM_PENALTIES_5_last_match_std,AWAY_TEAM_SUBSTITUTIONS_5_last_match_std,AWAY_TEAM_BALL_SAFE_5_last_match_std,AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_std,AWAY_TEAM_INJURIES_5_last_match_std,AWAY_TEAM_GOALS_5_last_match_std
ID,,,,,,,,,,,,,,,,,,,,,
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,5.0,5.0,4.0,0.0,6.0,8.0,4.0,3.0,2.0,3.0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,0.0,0.0,3.0,1.0,8.0,4.0,10.0,0.0,5.0,3.0
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,6.0,10.0,4.0,4.0,0.0,8.0,3.0,0.0,9.0,6.0
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,1.0,0.0,4.0,4.0,9.0,4.0,1.0,4.0,6.0,5.0
5,7.0,6.0,9.0,5.0,10.0,8.0,9.0,5.0,5.0,2.0,...,5.0,8.0,4.0,1.0,6.0,0.0,3.0,0.0,2.0,1.0


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data,train_scores, train_size=0.8, random_state=42)

# Train 80%, Test 20%

KeyboardInterrupt: 

In [33]:
# Entrainement par regression linéaire
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

# Predictions
y_pred_train = linear_regressor.predict(X_train)
y_pred_test = linear_regressor.predict(X_test)

# Evaluation
mse_train = mean_squared_error(y_train, y_pred_train)
score_train =  r2_score(y_train, y_pred_train)
print("R-squared:", score_train)
print("MSE train: ", mse_train)

#coeficients du modèle
a = linear_regressor.coef_[0][0]
b = linear_regressor.intercept_[0]
print(a)
print(b)

R-squared: 0.2272101479825357
MSE train:  0.16534646432857358
0.009233339887544451
0.31829034141742896


In [ ]:
#Test binaire
target = train_scores.loc[X_test.index].copy()

home_wins = target
home_wins = 0 * home_wins
home_wins.iloc[:,0] = 1

np.round(accuracy_score(home_wins,target),4)

np.float64(0.4355)

In [35]:
# Formatage [Home win, Draw, Away win]

predictions = y_pred_test
predictions = pd.DataFrame(predictions)

predictions[2] = 0
predictions.columns = [0,2,1]

# Applique un ranking pour choisir la prédiction la plus probable
predictions = (predictions.reindex(columns=[0,1,2]).rank(1,ascending=False)==1).astype(int).values

# Evalue avec accuracy_score
np.round(accuracy_score(predictions,target),4)

np.float64(0.4503)

In [39]:
test_data.head()

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,AWAY_TEAM_YELLOWCARDS_5_last_match_std,AWAY_TEAM_REDCARDS_5_last_match_std,AWAY_TEAM_OFFSIDES_5_last_match_std,AWAY_TEAM_ATTACKS_5_last_match_std,AWAY_TEAM_PENALTIES_5_last_match_std,AWAY_TEAM_SUBSTITUTIONS_5_last_match_std,AWAY_TEAM_BALL_SAFE_5_last_match_std,AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_std,AWAY_TEAM_INJURIES_5_last_match_std,AWAY_TEAM_GOALS_5_last_match_std
ID,,,,,,,,,,,,,,,,,,,,,
12303,3.0,6.0,5.0,1.0,3.0,9.0,8.0,10.0,6.0,7.0,...,6.0,0.0,1.0,4.0,0.0,8.0,7.0,2.0,7.0,0.0
12304,3.0,2.0,4.0,2.0,6.0,1.0,1.0,2.0,4.0,10.0,...,0.0,0.0,3.0,6.0,8.0,5.0,1.0,1.0,7.0,9.0
12305,7.0,10.0,8.0,8.0,1.0,4.0,5.0,0.0,10.0,4.0,...,10.0,0.0,0.0,2.0,0.0,4.0,2.0,3.0,0.0,9.0
12306,5.0,7.0,5.0,9.0,5.0,7.0,6.0,3.0,5.0,8.0,...,1.0,5.0,4.0,5.0,10.0,9.0,6.0,9.0,7.0,3.0
12307,5.0,4.0,6.0,3.0,5.0,0.0,0.0,2.0,2.0,9.0,...,3.0,0.0,6.0,5.0,0.0,4.0,3.0,10.0,3.0,1.0


In [ ]:
# On utilise le modèle sur les vrai données de test
test_data = test_data.fillna(0)
#Données manquantes remplacé par 0
y_pred = linear_regressor.predict(test_data)
y_pred = pd.DataFrame(y_pred)

# Formatage [Home win, Draw, Away win]
y_pred[2] = 0
y_pred.columns = [0,2,1]
# Applique un ranking pour choisir la prédiction la plus probable
y_pred = (y_pred.reindex(columns=[0,1,2]).rank(1,ascending=False)==1).astype(int).values

In [ ]:
# Nommage colonnes export fichier
y_pred = pd.DataFrame(y_pred, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])
y_pred.index = test_data.index
submission = y_pred.reset_index()
submission.to_csv('resultat_prediction.csv', index=False)